## AutoGen Distributed Runtimes

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

In [2]:
@dataclass
class Message:
    content: str

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper = Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")

In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some pros of using AutoGen for your new AI Agent project:

1. **Scalability**: AutoGen's modularity and extensibility allow for the creation of scalable systems that can grow with your project's requirements.

2. **Ease of Use**: It offers integrated observability and debugging tools, simplifying the monitoring and control of agent workflows. This reduces the complexity often associated with managing AI agents.

3. **Customization**: The framework supports customizable components, enabling developers to tailor solutions specific to their needs, enhancing flexibility in design.

4. **Efficiency**: AutoGen promotes efficient development processes, allowing teams to focus on innovation rather than repetitive tasks during AI agent orchestration.

5. **Robust Community Support**: Leveraging AutoGen may provide access to a community of developers and users for support, shared resources, and collaborative problem-solving.

Using AutoGen can lead to a more streamlined and effective development process, making it a strong choice for an AI Agent project. 

TERMINATE

## Cons of AutoGen:
Here are several cons or disadvantages of using AutoGen in an AI Agent project:

1. **Steep Learning Curve**: Users often find it challenging to grasp its evolving features and functionalities, making it difficult for new team members to become proficient quickly.

2. **User Difficulty**: Many report struggles in navigating and utilizing the platform effectively, which can hinder productivity and project progress.

3. **Lack of Intuition**: The interface may not be intuitive, requiring significant effort to understand how to use various features optimally.

4. **Inaccurate Responses**: There can be issues with the accuracy of generated outputs, which can undermine the reliability of the AI Agent's performance.

5. **Limited Features**: Some users feel that AutoGen may lack certain features they desire, which could restrict the capabilities of the AI solutions being developed.

6. **Poor Interface Design**: The overall user experience may suffer from inadequate interface design, making it less user-friendly.

7. **Need for UX Improvement**: There are suggestions that the user experience could greatly benefit from enhancements, indicating that the current design might not meet user needs effectively.

These factors should be carefully considered when making a decision about using AutoGen for your project. 

TERMINATE



## Decision:

Based on the research provided by the team, I would recommend using AutoGen for the AI Agent project. 

### Rationale:
The benefits of scalability, ease of use, customization, efficiency, and robust community support align well with the goals of creating a streamlined and effective development process. While the cons, including the learning curve and user interface issues, present valid concerns, the pros significantly contribute to the overall effectiveness and potential growth of the project. The advantages seem to outweigh the disadvantages and, with adequate training and support, the team can overcome the challenges presented by its learning curve and usability hurdles.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()